# Exploring Ocean Composites

When solving problems on a quantum computer, both pre-processing and post-processing can be largely helpful. Ocean provides several tools for both forms of processing that can be accessed through composites.

In this notebook, we explore several of these composites and consider ... [TODO]


## FixVariablesComposite

The first composite we'll look at is the `FixVariablesComposite`, which can be found in `dwave-preprocessing`.

In [1]:
from dwave.preprocessing import FixVariablesComposite

The `FixVariablesComposite` allows users to 'fix' some variables of a binary quadratic model (BQM) before it is passed to a sampler, guaranteeing that returned samples will include the fixed variables. 

By fixing variables, we are reducing the size of the problem that actually needs to be solved. This is especially helpful for larger problems that cannot be easily solved on the QPU. 

Here is the class signature:

In [2]:
?FixVariablesComposite

Using the FixVariablesComposite requires the user to specify a child sampler and the algorithm that determines which variables to fix. 

Let's demonstrate with a toy BQM of one ising variable.

In [3]:
import dimod
bqm = dimod.BinaryQuadraticModel.from_ising({'a':1}, {})

We'll use the `ExactSolver` from `dimod` as the child sampler, which is great for testing small problems because it calculates the energy for every possible sample.

In [4]:
from dimod import ExactSolver

exact_solver = ExactSolver()
sampler = FixVariablesComposite(exact_solver, algorithm='explicit')

By default, `algorithm` is set to `explicit`, meaning that the user must explicitly pass in a `fixed_variables` dict when sampling, as seen below:

In [5]:
# Using only the ExactSolver
sampleset = exact_solver.sample(bqm)
print(sampleset)

# Using the FixVariablesComposite and fixing the one variable to -1
sampleset = sampler.sample(bqm, fixed_variables={'a':-1})
print(sampleset)

   a energy num_oc.
0 -1   -1.0       1
1 +1    1.0       1
['SPIN', 2 rows, 2 samples, 1 variables]
   a energy num_oc.
0 -1   -1.0       1
['SPIN', 1 rows, 1 samples, 1 variables]


Instead of passing in the fixed variables explicitly, `algorithm` can also be set to `roof_duality`. With this algorithm, the composite will try to find minimizing assignments for some or all of the BQM's variables. For more information on the roof duality algorithm, see `roof_duality()` (TODO: add link to docs).

In [6]:
sampler = FixVariablesComposite(exact_solver, algorithm='roof_duality')
sampleset = sampler.sample(bqm)
print(sampleset)

strict:  True
fixed_variables:  {'a': -1}
   a energy num_oc.
0 -1   -1.0       1
['SPIN', 1 rows, 1 samples, 1 variables]


By default, we run the roof duality algorithm in `strict` mode. This means that we only fix variables when the variable assignments are True for all ground states. To demonstrate this, let's pick a different BQM:

In [7]:
bqm = dimod.BinaryQuadraticModel.from_ising({}, {'ab':-1})

This BQM has two ground states, either both `a` and `b` are -1 or both `a` and `b` are +1, which means that there are no variables that have a single value for all ground states. Thus, when `strict=True`, we don't have any fixed variables:

In [8]:
sampleset = sampler.sample(bqm, strict=True)
print(sampleset)

strict:  True
fixed_variables:  {}
   a  b energy num_oc.
0 -1 -1   -1.0       1
2 +1 +1   -1.0       1
1 +1 -1    1.0       1
3 -1 +1    1.0       1
['SPIN', 4 rows, 4 samples, 2 variables]


When `strict=False`, we also fix the variables with assignments that are True for some but not all ground states.

In [9]:
sampleset = sampler.sample(bqm, strict=False)
print(sampleset)

strict:  False
fixed_variables:  {'a': -1, 'b': -1}
   a  b energy num_oc.
0 -1 -1   -1.0       1
['SPIN', 1 rows, 1 samples, 2 variables]


Now let's try the composite out on a more difficult problem. In this next example, we'll call the QPU using `DWaveSampler` as our child sampler. 

In [10]:
from dwave.system import DWaveSampler, DWaveCliqueSampler, EmbeddingComposite
qpu_sampler = EmbeddingComposite(DWaveSampler())
sampler = FixVariablesComposite(qpu_sampler, algorithm='roof_duality')

Using one of `dimod`'s generators, we create a BQM for an anti-crossing problem with a single clique of 20 variables.

In [34]:
import dimod
bqm = dimod.generators.anti_crossing_clique(20)

Let's compare our results with and without using the `FixVariablesComposite`.

In [35]:
sampleset = qpu_sampler.sample(bqm, num_reads=10, label='D-Wave 101: Ocean Composites')
print(sampleset)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 19 energy num_oc. ...
0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 ... -1  -54.0       1 ...
1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 +1 +1 +1 -1 ... -1  -54.0       1 ...
2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ... -1  -54.0       1 ...
3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 ... +1  -54.0       1 ...
4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 -1 +1 ... -1  -54.0       1 ...
5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 -1 ... +1  -54.0       1 ...
6 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 +1 ... -1  -54.0       1 ...
7 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 +1 -1 -1 +1 ... +1  -54.0       1 ...
8 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 +1 +1 +1 ... +1  -54.0       1 ...
9 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 -1 -1 +1 +1 -1 ... -1  -54.0       1 ...
['SPIN', 10 rows, 10 samples, 20 variables]


In [36]:
fix_sampleset = sampler.sample(bqm, num_reads=10, strict=False, label='D-Wave 101: Ocean Composites')
print(fix_sampleset)

strict:  False
fixed_variables:  {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1}
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 energy num_oc.
0 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1  -56.0       1
['SPIN', 1 rows, 1 samples, 20 variables]


In [ ]:
TODO: maybe add a plot for comparison?

## SpinReversalTransformComposite

The QPU is not perfect and qubits can be slightly biased to one direction or another. To reduce the impact of these unintended biases and potentially improve results, users can apply spin reversal transforms (also known as 'gauge transforms') on their problem before solving. The `DWaveSampler` has a `num_spin_reversal_transforms` parameter than allows users to specify the number of transforms that are applied to their problem from the server-side. 

Alternatively, users may use the `SpinReversalTransformComposite`, another pre-processing composite that can be found in `dwave-preprocessing`. 

In [37]:
from dwave.preprocessing import SpinReversalTransformComposite

This composite applies the spin reversal transforms by flipping some variables, passing them to the child sampler, and then flipping the same variables back in the returned samples. Each variable has a 50% chance of being flipped. Increasing the number of transforms can improve results, but may also increase the run time.

In [58]:
from neal import SimulatedAnnealingSampler
# neal_sampler = SimulatedAnnealingSampler()
qpu_sampler = EmbeddingComposite(DWaveSampler())
sampler = SpinReversalTransformComposite(qpu_sampler)

In [62]:
# bqm = dimod.generators.random.ran_r(1,25)
bqm = dimod.generators.frustrated_loop(20, 10)
sampleset = qpu_sampler.sample(bqm, num_reads=10, label='D-Wave 101: Ocean Composites')
print(sampleset)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 19 energy num_oc. ...
0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 +1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
3 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 +1 -1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
4 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       2 ...
5 -1 -1 +1 -1 -1 -1 -1 -1 +1 -1 +1 -1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
6 +1 -1 -1 -1 -1 -1 -1 -1 +1 -1 +1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
7 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
8 +1 -1 +1 -1 -1 +1 -1 -1 +1 +1 +1 -1 -1 +1 -1 -1 +1 ... +1  -21.0       1 ...
['SPIN', 9 rows, 10 samples, 20 variables]


In [63]:
sampleset = sampler.sample(bqm, num_spin_reversal_transforms=10, label='D-Wave 101: Ocean Composites')
print(sampleset)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 19 energy num_oc. ...
2 -1 +1 +1 +1 +1 +1 +1 +1 +1 +1 -1 +1 +1 +1 +1 +1 +1 ... +1  -29.0       1 ...
3 -1 +1 -1 +1 +1 +1 +1 +1 -1 +1 -1 +1 +1 +1 +1 +1 +1 ... +1  -29.0       1 ...
4 +1 +1 +1 +1 +1 +1 +1 +1 -1 +1 -1 -1 +1 +1 +1 +1 +1 ... +1  -29.0       1 ...
6 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
7 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
8 +1 +1 +1 +1 +1 +1 +1 +1 -1 +1 -1 -1 +1 +1 +1 +1 +1 ... +1  -29.0       1 ...
9 -1 -1 -1 -1 -1 -1 -1 -1 +1 -1 -1 +1 -1 -1 -1 -1 -1 ... -1  -29.0       1 ...
0 +1 -1 -1 +1 -1 +1 -1 -1 -1 +1 +1 -1 -1 -1 -1 -1 +1 ... -1  -25.0       1 ...
1 +1 +1 -1 -1 +1 +1 +1 -1 -1 +1 +1 +1 +1 +1 -1 -1 +1 ... +1  -25.0       1 ...
5 -1 -1 +1 +1 -1 -1 -1 -1 +1 -1 +1 -1 -1 -1 +1 -1 -1 ... -1  -25.0       1 ...
['SPIN', 10 rows, 10 samples, 20 variables]
